Make sure to put the train.csv and test.csv in a folder called data in the home directory.

Ankith: Neural network

Jenny: Logistic Regression

Vedu: Random Forest

Ethan: Support Vector Machines

Vidit: k-Nearest Neighbors

In [100]:
%pylab inline
# Pandas is a nice utilitiy that enables some easy data manipulation, especially from a csv
import pandas as pd
# Numpy lets us work with arrays
import numpy as np
import random
# Sklearn provides various modules with a common API
from sklearn import svm, tree, neighbors, neural_network
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\ankit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [101]:
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

In [102]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [103]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [104]:
print(train_data.columns[train_data.isna().any()].tolist())

['Age', 'Cabin', 'Embarked']


In [105]:
random.seed(100)
np.random.seed(100)

train_data.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)
train_data.set_index(keys=['PassengerId'], drop=True, inplace=True)

test_data.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)
test_data.set_index(keys=['PassengerId'], drop=True, inplace=True)

train_nan_map = {'Age': train_data['Age'].mean(), 'Fare': train_data['Fare'].mean(), 'Embarked': train_data['Embarked'].mode()[0]}
test_nan_map = {'Age': test_data['Age'].mean(), 'Fare': test_data['Fare'].mean(), 'Embarked': test_data['Embarked'].mode()[0]}

train_data.fillna(value=train_nan_map, inplace=True)
test_data.fillna(value=test_nan_map, inplace=True)

columns_map = {'Embarked': {'C': 0, 'Q': 1, 'S': 2}, 'Sex': {'male': 0, 'female': 1}}
train_data.replace(columns_map, inplace=True)
test_data.replace(columns_map, inplace=True)

X_train = train_data.loc[:, train_data.columns != 'Survived']
y_train = train_data.loc[:, 'Survived']

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=10)

In [106]:
print(X_train.head())
print(y_train.head())
print(test_data.head())
print(f"Test Mean: {y_test.mean()}")
print(f"Train Mean: {y_train.mean()}")

             Pclass  Sex        Age  SibSp  Parch    Fare  Embarked
PassengerId                                                        
464               2    0  48.000000      0      0  13.000         2
160               3    0  29.699118      8      2  69.550         2
48                3    1  29.699118      0      0   7.750         1
403               3    1  21.000000      1      0   9.825         2
619               2    1   4.000000      2      1  39.000         2
PassengerId
464    0
160    0
48     1
403    0
619    1
Name: Survived, dtype: int64
             Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
PassengerId                                                    
892               3    0  34.5      0      0   7.8292         1
893               3    1  47.0      1      0   7.0000         2
894               2    0  62.0      0      0   9.6875         1
895               3    0  27.0      0      0   8.6625         2
896               3    1  22.0      1      1  12.2875 

In [107]:
train_data.columns[train_data.isna().any()].tolist()
train_data[train_data['Embarked'].isna() == True]
train_data.describe(include='all')

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,29.699118,0.523008,0.381594,32.204208,1.536476
std,0.486592,0.836071,0.477990,13.002015,1.102743,0.806057,49.693429,0.791503
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400,1.000000
50%,0.000000,3.000000,0.000000,29.699118,0.000000,0.000000,14.454200,2.000000
75%,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000,2.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


In [108]:
passengerids = X_test.index.to_list()
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
464,2,0,48.000000,0,0,13.000,2
160,3,0,29.699118,8,2,69.550,2
48,3,1,29.699118,0,0,7.750,1
403,3,1,21.000000,1,0,9.825,2
619,2,1,4.000000,2,1,39.000,2


In [109]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


random.seed(12)
np.random.seed(12)
MLP_cls = MLPClassifier(hidden_layer_sizes=(10,10,10,10), max_iter = 200)
MLP_cls.fit(X_train,y_train)
y_pred = MLP_cls.predict(test_data.values)
print(MLP_cls.score(X_test,y_test))

0.847457627118644


C:\Users\ankit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [110]:
confusion_matrix = confusion_matrix(y_test,MLP_cls.predict(X_test))
class_names=['0','1']
plt.figure()

ConfusionMatrixDisplay(confusion_matrix)
plt.show()

print(confusion_matrix)
FNR = confusion_matrix[1][0] / (confusion_matrix[1][0] + confusion_matrix[1][1])
FPR = confusion_matrix[0][1] / (confusion_matrix[0][0] + confusion_matrix[0][1])
print(FNR,FPR)

<Figure size 640x480 with 0 Axes>

[[179  12]
 [ 33  71]]
0.3173076923076923 0.06282722513089005


In [111]:

def save_predictions_to_csv(predictions: list, filename: str = "titanic_prediction_group_4_ankith.csv") -> None:
    """
    Save the predictions to a CSV file.

    Args:
    - predictions (list): List of predictions to save.
    - filename (str, optional): Name of the CSV file to save to. Defaults to "titanic_predictions_group_4_ankith.csv".

    Returns:
    None
    """
    df = pd.DataFrame(predictions, columns=["PassengerID"])
    df.to_csv(filename, index=False)

In [112]:


#save_predictions_to_csv(passengerids,"passengerids.csv")
passid_prediction = {'PassengerId': test_data.index, 'Survived': y_pred}
df_predictions = pd.DataFrame(passid_prediction)
df_predictions.to_csv("titanic_predictions_ankith_MLP_classifier.csv", index=False)